In [1]:
import csv
import time
import math

# Leer base de datos desde CSV

csvName = "database1m.csv"

database = []
with open(csvName, newline="") as f:
    reader = csv.DictReader(f)
    for row in reader:
        database.append(int(row["valor"]))

print(f"Registros cargados: {len(database)}")

N = len(database)

# Elegir un valor aleatorio del archivo
#target = random.choice(database)
#print(f"Elemento a buscar: {target}")

#cambiar segun busqueda
target = 1405556



Registros cargados: 1000000


In [2]:
import numpy as np
from qiskit import QuantumCircuit, transpile
from qiskit_aer import Aer
from qiskit.visualization import plot_histogram

# Calcular número de qubits necesarios
n_qubits = math.ceil(math.log2(N))
print(f"Número de qubits necesarios: {n_qubits} (2^{n_qubits} = {2**n_qubits} ≥ {N})")

# Convertir el valor objetivo a binario compatible con el número de qubits
target_index = (target - 1) % (2**n_qubits)
target_state = format(target_index, f'0{n_qubits}b')
print(f"Estado objetivo: {target_state}")

def grover_oracle(n, target):
    qc = QuantumCircuit(n)
    for i, bit in enumerate(target):
        if bit == '0':
            qc.x(i)
    qc.h(n - 1)
    qc.mcx(list(range(n - 1)), n - 1)
    qc.h(n - 1)
    for i, bit in enumerate(target):
        if bit == '0':
            qc.x(i)
    return qc.to_gate(label="Oracle")

def grover_diffuser(n):
    qc = QuantumCircuit(n)
    qc.h(range(n))
    qc.x(range(n))
    qc.h(n - 1)
    qc.mcx(list(range(n - 1)), n - 1)
    qc.h(n - 1)
    qc.x(range(n))
    qc.h(range(n))
    return qc.to_gate(label="Diffuser")


Número de qubits necesarios: 20 (2^20 = 1048576 ≥ 1000000)
Estado objetivo: 01010111001001110011


In [5]:
qc = QuantumCircuit(n_qubits, n_qubits)

# Superposición inicial
qc.h(range(n_qubits))

oracle = grover_oracle(n_qubits, target_state)
diffuser = grover_diffuser(n_qubits)

# Número óptimo de iteraciones
iterations = int(np.floor(np.pi / 4 * np.sqrt(2**n_qubits)))
print(f"Iteraciones de Grover: {iterations}")

for _ in range(iterations):
    qc.append(oracle, range(n_qubits))
    qc.append(diffuser, range(n_qubits))

qc.measure(range(n_qubits), range(n_qubits))
#qc.draw('mpl')


Iteraciones de Grover: 804


In [4]:
sim = Aer.get_backend('qasm_simulator')
compiled = transpile(qc, sim)

start = time.perf_counter()
result = sim.run(compiled, shots=256).result()
end = time.perf_counter()

counts = result.get_counts()
plot_histogram(counts)

most_probable = max(counts, key=counts.get)
print(f"\nEstado más probable: {most_probable}")
print(f"Tiempo simulación cuántica: {end - start:.6f} segundos")

if most_probable == target_state:
    print("✅ Grover encontró el elemento correcto.")
else:
    print("⚠️ El resultado puede variar por ruido o simulación limitada.")



Estado más probable: 11001110010011101010
Tiempo simulación cuántica: 17.306381 segundos
⚠️ El resultado puede variar por ruido o simulación limitada.
